In [ ]:
import json
from micromort.data_stores.mongodb import getConnection
from multiprocessing.dummy import Pool as ThreadPool 
import traceback, sys
import os

In [ ]:
mongo_db_name = "micromort"
mongo_collection_name = "newstweets"
mongoClient = getConnection(mongo_db_name, mongo_collection_name)

In [ ]:
def gen_dict_extract(key, var):
    if hasattr(var, 'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in gen_dict_extract(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in gen_dict_extract(key, d):
                        yield result

def getUrl(tweet):
    return gen_dict_extract("expanded_url", tweet)

In [ ]:
count = 0
fil_path = ""
f = open("./my_data.txt", "a")
def myFun(tweet):
    try:
        #idx = multiprocessing.current_process()
        #print(idx, os.getpid())
        _str = ""
        global count
        count = count+1
        if count%1000 == 0:
            print(count/1000)

        urls = getUrl(tweet)
        tweet_id = tweet["id"]
        filtered_urls = []
        for url in urls:
            if "twitter.com" not in url:
                filtered_urls.append(url)

        if not len(filtered_urls):
            return 1

        for url in filtered_urls:
            url = url[:1000]
            #url_id = insertUrl(url, con)
            _str += str(tweet_id) + "\t" + url + "\n"
        f.write(_str)
        return 1
    except Exception as e:
        print(e)
        traceback.print_exc()
        return 1
    

pool = ThreadPool(6) 
results = pool.imap(myFun, mongoClient.find())    

In [ ]:
cat  my_data.txt | awk '{print $2}' | sort | uniq > unique_urls